In [7]:
import requests
import os
import time
import os
import pandas as pd
import zipfile

# Define directories
data_dir = 'data/aemo_data'
out_dir = 'data/concatenated_data'
analysis_dir = 'data/analysis'
os.makedirs(out_dir, exist_ok=True)
# Create directory to store files
os.makedirs(data_dir, exist_ok=True)
os.makedirs(analysis_dir, exist_ok=True)


# List of states/regions
regions = ['NSW1', 'QLD1', 'SA1', 'TAS1', 'VIC1']




## Price

In [2]:
# Base URL template
base_url = "https://aemo.com.au/aemo/data/nem/priceanddemand/PRICE_AND_DEMAND_{}_{}.csv"

# Headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Loop through years and months
for year in range(2019, 2025):
    for month in range(1, 13):
        month_str = f"{year}{month:02d}"
        if year == 2024 and month > 12:
            continue
        for region in regions:
            filename = f"data/aemo_data/PRICE_AND_DEMAND_{month_str}_{region}.csv"
            # Skip if file already exists
            if os.path.exists(filename):
                print(f"Skipping {filename} - already exists")
                continue
                
            url = base_url.format(month_str, region)
            try:
                response = requests.get(url, headers=headers)
                response.raise_for_status()
                with open(filename, 'wb') as f:
                    f.write(response.content)
                print(f"Downloaded {filename}")
            except requests.exceptions.RequestException as e:
                print(f"Failed to download {url}: {e}")
            time.sleep(1)  # Wait 1 second between requests

Downloaded data/aemo_data/PRICE_AND_DEMAND_201901_NSW1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201901_QLD1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201901_SA1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201901_TAS1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201901_VIC1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201902_NSW1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201902_QLD1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201902_SA1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201902_TAS1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201902_VIC1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201903_NSW1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201903_QLD1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201903_SA1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201903_TAS1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201903_VIC1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201904_NSW1.csv
Downloaded data/aemo_data/PRICE_AND_DEMAND_201904_QLD1.csv


In [3]:
def concat_files_by_year_and_region():
    for year in range(2019, 2025):
        for region in regions:
            # Collect all files for the given year and region
            region_files = [
                os.path.join(data_dir, file) 
                for file in os.listdir(data_dir) 
                if file.startswith(f"PRICE_AND_DEMAND_{year}") and file.endswith(f"_{region}.csv")
            ]

            # If no files found, skip
            if not region_files:
                print(f"No files found for {year} - {region}")
                continue

            # Load and concatenate CSV files
            data_frames = []
            for file in region_files:
                try:
                    df = pd.read_csv(file)
                    data_frames.append(df)
                except Exception as e:
                    print(f"Failed to read {file}: {e}")

            # Concatenate and save the result
            if data_frames:
                concatenated_df = pd.concat(data_frames, ignore_index=True)
                output_filename = f"PRICE_AND_DEMAND_{year}_{region}.csv"
                output_path = os.path.join(out_dir, output_filename)
                concatenated_df.to_csv(output_path, index=False)
                print(f"Concatenated data saved to {output_path}")

In [4]:
concat_files_by_year_and_region()

Concatenated data saved to data/concatenated_data\PRICE_AND_DEMAND_2019_NSW1.csv
Concatenated data saved to data/concatenated_data\PRICE_AND_DEMAND_2019_QLD1.csv
Concatenated data saved to data/concatenated_data\PRICE_AND_DEMAND_2019_SA1.csv
Concatenated data saved to data/concatenated_data\PRICE_AND_DEMAND_2019_TAS1.csv
Concatenated data saved to data/concatenated_data\PRICE_AND_DEMAND_2019_VIC1.csv
Concatenated data saved to data/concatenated_data\PRICE_AND_DEMAND_2020_NSW1.csv
Concatenated data saved to data/concatenated_data\PRICE_AND_DEMAND_2020_QLD1.csv
Concatenated data saved to data/concatenated_data\PRICE_AND_DEMAND_2020_SA1.csv
Concatenated data saved to data/concatenated_data\PRICE_AND_DEMAND_2020_TAS1.csv
Concatenated data saved to data/concatenated_data\PRICE_AND_DEMAND_2020_VIC1.csv
Concatenated data saved to data/concatenated_data\PRICE_AND_DEMAND_2021_NSW1.csv
Concatenated data saved to data/concatenated_data\PRICE_AND_DEMAND_2021_QLD1.csv
Concatenated data saved to dat

In [5]:
def concat_files_by_year():
    for region in regions:
        # Collect all files for the region across all years
        region_files = [
            os.path.join(data_dir, file) 
            for file in os.listdir(data_dir) 
            if file.endswith(f"_{region}.csv")
        ]

        # If no files found, skip
        if not region_files:
            print(f"No files found for {region}")
            continue

        # Load and concatenate CSV files
        data_frames = []
        for file in region_files:
            try:
                df = pd.read_csv(file)
                data_frames.append(df)
            except Exception as e:
                print(f"Failed to read {file}: {e}")

        # Concatenate and save the result
        if data_frames:
            concatenated_df = pd.concat(data_frames, ignore_index=True)
            # Sort by date to ensure chronological order
            concatenated_df['SETTLEMENTDATE'] = pd.to_datetime(concatenated_df['SETTLEMENTDATE'])
            concatenated_df = concatenated_df.sort_values('SETTLEMENTDATE')

            # split 'SETTLEMENTDATE' to year, month, day, hour, minute, and weekday
            concatenated_df['YEAR'] = concatenated_df['SETTLEMENTDATE'].dt.year
            concatenated_df['MONTH'] = concatenated_df['SETTLEMENTDATE'].dt.month
            concatenated_df['DAY'] = concatenated_df['SETTLEMENTDATE'].dt.day
            concatenated_df['HOUR'] = concatenated_df['SETTLEMENTDATE'].dt.hour
            concatenated_df['MINUTE'] = concatenated_df['SETTLEMENTDATE'].dt.minute
            concatenated_df['WEEKDAY'] = concatenated_df['SETTLEMENTDATE'].dt.weekday
            
            output_filename = f"PRICE_AND_DEMAND_ALL_YEARS_{region}.csv"
            output_path = os.path.join(out_dir, output_filename)
            concatenated_df.to_csv(output_path, index=False)
            print(f"Concatenated data for all years saved to {output_path}")

concat_files_by_year()

Concatenated data for all years saved to data/concatenated_data\PRICE_AND_DEMAND_ALL_YEARS_NSW1.csv
Concatenated data for all years saved to data/concatenated_data\PRICE_AND_DEMAND_ALL_YEARS_QLD1.csv
Concatenated data for all years saved to data/concatenated_data\PRICE_AND_DEMAND_ALL_YEARS_SA1.csv
Concatenated data for all years saved to data/concatenated_data\PRICE_AND_DEMAND_ALL_YEARS_TAS1.csv
Concatenated data for all years saved to data/concatenated_data\PRICE_AND_DEMAND_ALL_YEARS_VIC1.csv


In [6]:
# calculate the mean, median, min, max of the trading price by month of years (201901, 201902, ..., 202412)

def calculate_price_stats_by_month():
    for region in regions:
        input_filename = f"PRICE_AND_DEMAND_ALL_YEARS_{region}.csv"
        input_path = os.path.join(out_dir, input_filename)
        output_filename = f"PRICE_STATS_BY_MONTH_{region}.csv"
        output_path = os.path.join(analysis_dir, output_filename)

        try:
            df = pd.read_csv(input_path)
            df['SETTLEMENTDATE'] = pd.to_datetime(df['SETTLEMENTDATE'])
            df['YEAR_MONTH'] = df['SETTLEMENTDATE'].dt.strftime('%Y-%m')
            price_stats = df.groupby('YEAR_MONTH').agg(
                RRP_mean=('RRP', 'mean'),
                RRP_median=('RRP', 'median'),
                RRP_min=('RRP', 'min'),
                RRP_max=('RRP', 'max'),
                TOTALDEMAND_mean=('TOTALDEMAND', 'mean'),
                TOTALDEMAND_median=('TOTALDEMAND', 'median'),
                TOTALDEMAND_min=('TOTALDEMAND', 'min'),
                TOTALDEMAND_max=('TOTALDEMAND', 'max')
            ).round(2)
            #split 'YEAR_MONTH' to year and month
            price_stats['YEAR'] = price_stats.index.str.split('-').str[0]
            price_stats['MONTH'] = price_stats.index.str.split('-').str[1]

            # remove 2025 year data
            price_stats = price_stats[price_stats['YEAR'] != '2025']

            price_stats.to_csv(output_path)
            print(f"Price statistics by month saved to {output_path}")
        except Exception as e:
            print(f"Failed to calculate price statistics for {region}: {e}")
        

calculate_price_stats_by_month()

Price statistics by month saved to data/analysis\PRICE_STATS_BY_MONTH_NSW1.csv
Price statistics by month saved to data/analysis\PRICE_STATS_BY_MONTH_QLD1.csv
Price statistics by month saved to data/analysis\PRICE_STATS_BY_MONTH_SA1.csv
Price statistics by month saved to data/analysis\PRICE_STATS_BY_MONTH_TAS1.csv
Price statistics by month saved to data/analysis\PRICE_STATS_BY_MONTH_VIC1.csv


In [7]:
def calculate_price_stats_by_week():
    for region in regions:
        input_filename = f"PRICE_AND_DEMAND_ALL_YEARS_{region}.csv"
        input_path = os.path.join(out_dir, input_filename)
        output_filename = f"PRICE_STATS_BY_WEEK_{region}.csv"
        output_path = os.path.join(analysis_dir, output_filename)

        try:
            df = pd.read_csv(input_path)
            df['SETTLEMENTDATE'] = pd.to_datetime(df['SETTLEMENTDATE'])
            df['YEAR_WEEK'] = df['SETTLEMENTDATE'].dt.strftime('%Y-%U')  # Year-Week format
            price_stats = df.groupby('YEAR_WEEK').agg(
                RRP_mean=('RRP', 'mean'),
                RRP_median=('RRP', 'median'),
                RRP_min=('RRP', 'min'),
                RRP_max=('RRP', 'max'),
                TOTALDEMAND_mean=('TOTALDEMAND', 'mean'),
                TOTALDEMAND_median=('TOTALDEMAND', 'median'),
                TOTALDEMAND_min=('TOTALDEMAND', 'min'),
                TOTALDEMAND_max=('TOTALDEMAND', 'max')
            ).round(2)
            #split 'YEAR_WEEK' to year and week
            price_stats['YEAR'] = price_stats.index.str.split('-').str[0]
            price_stats['WEEK'] = price_stats.index.str.split('-').str[1]
            price_stats = price_stats[price_stats['YEAR'] != '2025']

            price_stats.to_csv(output_path)
            print(f"Price statistics by week saved to {output_path}")
        except Exception as e:
            print(f"Failed to calculate price statistics for {region}: {e}")

calculate_price_stats_by_week()

Price statistics by week saved to data/analysis\PRICE_STATS_BY_WEEK_NSW1.csv
Price statistics by week saved to data/analysis\PRICE_STATS_BY_WEEK_QLD1.csv
Price statistics by week saved to data/analysis\PRICE_STATS_BY_WEEK_SA1.csv
Price statistics by week saved to data/analysis\PRICE_STATS_BY_WEEK_TAS1.csv
Price statistics by week saved to data/analysis\PRICE_STATS_BY_WEEK_VIC1.csv


In [8]:
def calculate_price_stats_by_day():
    for region in regions:
        input_filename = f"PRICE_AND_DEMAND_ALL_YEARS_{region}.csv"
        input_path = os.path.join(out_dir, input_filename)
        output_filename = f"PRICE_STATS_BY_DAY_{region}.csv"
        output_path = os.path.join(analysis_dir, output_filename)

        try:
            df = pd.read_csv(input_path)
            df['SETTLEMENTDATE'] = pd.to_datetime(df['SETTLEMENTDATE'])
            df['YEAR_MONTH_DAY'] = df['SETTLEMENTDATE'].dt.strftime('%Y-%m-%d')
            price_stats = df.groupby('YEAR_MONTH_DAY').agg(
                RRP_mean=('RRP', 'mean'),
                RRP_median=('RRP', 'median'),
                RRP_min=('RRP', 'min'),
                RRP_max=('RRP', 'max'),
                TOTALDEMAND_mean=('TOTALDEMAND', 'mean'),
                TOTALDEMAND_median=('TOTALDEMAND', 'median'),
                TOTALDEMAND_min=('TOTALDEMAND', 'min'),
                TOTALDEMAND_max=('TOTALDEMAND', 'max')
            ).round(2)
            #split 'YEAR_MONTH_DAY' to year, month, and day
            price_stats['YEAR'] = price_stats.index.str.split('-').str[0]
            price_stats['MONTH'] = price_stats.index.str.split('-').str[1]
            price_stats['DAY'] = price_stats.index.str.split('-').str[2]
            price_stats['WEEKDAY'] = pd.to_datetime(price_stats.index).dayofweek
            price_stats = price_stats[price_stats['YEAR'] != '2025']

            price_stats.to_csv(output_path)
            print(f"Price statistics by day saved to {output_path}")
        except Exception as e:
            print(f"Failed to calculate price statistics for {region}: {e}")

calculate_price_stats_by_day()

Price statistics by day saved to data/analysis\PRICE_STATS_BY_DAY_NSW1.csv
Price statistics by day saved to data/analysis\PRICE_STATS_BY_DAY_QLD1.csv
Price statistics by day saved to data/analysis\PRICE_STATS_BY_DAY_SA1.csv
Price statistics by day saved to data/analysis\PRICE_STATS_BY_DAY_TAS1.csv
Price statistics by day saved to data/analysis\PRICE_STATS_BY_DAY_VIC1.csv


In [9]:
def calculate_price_stats_by_hour():
    for region in regions:
        input_filename = f"PRICE_AND_DEMAND_ALL_YEARS_{region}.csv"
        input_path = os.path.join(out_dir, input_filename)
        output_filename = f"PRICE_STATS_BY_HOUR_{region}.csv"
        output_path = os.path.join(analysis_dir, output_filename)

        try:
            df = pd.read_csv(input_path)
            df['SETTLEMENTDATE'] = pd.to_datetime(df['SETTLEMENTDATE'])
            df['YEAR_MONTH_DAY_HOUR'] = df['SETTLEMENTDATE'].dt.strftime('%Y-%m-%d %H:00')
            price_stats = df.groupby('YEAR_MONTH_DAY_HOUR').agg(
                RRP_mean=('RRP', 'mean'),
                RRP_median=('RRP', 'median'),
                RRP_min=('RRP', 'min'),
                RRP_max=('RRP', 'max'),
                TOTALDEMAND_mean=('TOTALDEMAND', 'mean'),
                TOTALDEMAND_median=('TOTALDEMAND', 'median'),
                TOTALDEMAND_min=('TOTALDEMAND', 'min'),
                TOTALDEMAND_max=('TOTALDEMAND', 'max')
            ).round(2)
            #split 'YEAR_MONTH_DAY_HOUR' to year, month, day, and hour
            price_stats['YEAR'] = price_stats.index.str.split('-').str[0]
            price_stats['MONTH'] = price_stats.index.str.split('-').str[1]
            price_stats['DAY'] = price_stats.index.str.split('-').str[2]
            price_stats['HOUR'] = price_stats.index.str.split(' ').str[1].str.split(':').str[0]
            price_stats['WEEKDAY'] = pd.to_datetime(price_stats.index).dayofweek
            price_stats = price_stats[price_stats['YEAR'] != '2025']

            price_stats.to_csv(output_path)
            print(f"Price statistics by hour saved to {output_path}")
        except Exception as e:
            print(f"Failed to calculate price statistics for {region}: {e}")

calculate_price_stats_by_hour()

Price statistics by hour saved to data/analysis\PRICE_STATS_BY_HOUR_NSW1.csv
Price statistics by hour saved to data/analysis\PRICE_STATS_BY_HOUR_QLD1.csv
Price statistics by hour saved to data/analysis\PRICE_STATS_BY_HOUR_SA1.csv
Price statistics by hour saved to data/analysis\PRICE_STATS_BY_HOUR_TAS1.csv
Price statistics by hour saved to data/analysis\PRICE_STATS_BY_HOUR_VIC1.csv


## Outage

In [4]:

# Headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [6]:
# Download the latest data of NETWORK.NETWORK_OUTAGEDETAIL in MMS Data Model
# NETWORK.NETWORK_OUTAGEDETAIL stores the details of network outages in the NEM from 2003 to the present
# there is no need to download the table for each year and month
# The latest data can be found from the following URL: https://visualisations.aemo.com.au/aemo/nemweb/#mms-data-model

# 2025 January data
outage_detail_url = "https://nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/2025/MMSDM_2025_01/MMSDM_Historical_Data_SQLLoader/DATA/PUBLIC_ARCHIVE%23NETWORK_OUTAGEDETAIL%23FILE01%23202501010000.zip"

# Download the zip file
outage_detail_zipname = "data/aemo_data/NETWORK_OUTAGEDETAIL_202501.zip"
outage_detail_csvname = "PUBLIC_ARCHIVE#NETWORK_OUTAGEDETAIL#FILE01#202501010000.CSV"

try:
    response = requests.get(outage_detail_url, headers=headers)
    response.raise_for_status()
    with open(outage_detail_zipname, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {outage_detail_zipname}")

    # Unzip the file
    import zipfile
    with zipfile.ZipFile(outage_detail_zipname, 'r') as zip_ref:
        zip_ref.extractall("data/aemo_data")
    print(f"Unzipped {outage_detail_zipname}")
except requests.exceptions.RequestException as e:
    print(f"Failed to download {outage_detail_url}: {e}")

Downloaded data/aemo_data/NETWORK_OUTAGEDETAIL_202501.zip
Unzipped data/aemo_data/NETWORK_OUTAGEDETAIL_202501.zip


In [9]:
# prepare the data for analysis

## remove first line and last line of the csv file
## STARTTIME,ENDTIME,SUBMITTEDDATE - "2003/03/15 07:00:00","2003/06/13 17:00:00","2003/04/29 12:13:51"
## filter data for 2022 and later using START_DATE
## save the prepared data to data/analysis/NETWORK_OUTAGEDETAIL.csv
## gzip the file

def prepare_outage_detail_data():
    input_filename = "data/aemo_data/PUBLIC_ARCHIVE#NETWORK_OUTAGEDETAIL#FILE01#202501010000.CSV"
    output_filename = "data/analysis/NETWORK_OUTAGEDETAIL_202201_202501.csv"

    try:
        df = pd.read_csv(input_filename, skiprows=1, skipfooter=1, engine='python')
        df['STARTTIME'] = pd.to_datetime(df['STARTTIME'])
        df = df[df['STARTTIME'].dt.year >= 2022]
        df.to_csv(output_filename, index=False)
        print(f"Prepared data saved to {output_filename}")
    except Exception as e:
        print(f"Failed to prepare data: {e}")

prepare_outage_detail_data()

Prepared data saved to data/analysis/NETWORK_OUTAGEDETAIL_202201_202501.csv.gz


## Renewable Energy

In [2]:

# Headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

NEM Registration

In [3]:
## Dataset 1: NEM Registration and Exemption List.xlsx
## The NEM Registration and Exemption List is a list of all registered participants in the NEM and their registration details.

## Download the latest data of NEM Registration and Exemption List
## The latest data can be found from the following URL: 
## https://www.aemo.com.au/-/media/files/electricity/nem/participant_information/nem-registration-and-exemption-list.xlsx?

## Download the file
registration_url = "https://www.aemo.com.au/-/media/files/electricity/nem/participant_information/nem-registration-and-exemption-list.xlsx"
registration_filename = "data/aemo_data/NEM_Registration_and_Exemption_List.xlsx"

try:
    response = requests.get(registration_url, headers=headers)
    response.raise_for_status()
    with open(registration_filename, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {registration_filename}")
except requests.exceptions.RequestException as e:
    print(f"Failed to download {registration_url}: {e}")


Downloaded data/aemo_data/NEM_Registration_and_Exemption_List.xlsx


In [4]:
## prepare the data for analysis
## load the excel file, extract the 'PU and Scheduled Loads' sheet
## save the data to data/analysis/NEM_Registration.csv

def prepare_registration_data():
    input_filename = "data/aemo_data/NEM_Registration_and_Exemption_List.xlsx"
    output_filename = "data/analysis/NEM_Registration.csv"

    try:
        df = pd.read_excel(input_filename, sheet_name='PU and Scheduled Loads')
        df.to_csv(output_filename, index=False)
        print(f"Prepared data saved to {output_filename}")
    except Exception as e:
        print(f"Failed to prepare data: {e}")

prepare_registration_data()

Prepared data saved to data/analysis/NEM_Registration.csv


In [19]:
## Analysis NEM Registration data
# load the data from data/analysis/NEM_Registration.csv
nem_registration = pd.read_csv("data/analysis/NEM_Registration.csv")
nem_registration.head()

,Participant,Station Name,Region,Dispatch Type,Category,Classification,Fuel Source - Primary,Fuel Source - Descriptor,Technology Type - Primary,Technology Type - Descriptor,Units,Aggregation,DUID,Reg Cap generation (MW),Max Cap generation (MW),Max ROC/Min generation,Reg Cap consumption (MW),Max Cap consumption (MW),Max ROC/Min consumption,Comments
0,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,1-3,Y,ADPBA1G,7.76,6.15,2,NaN,NaN,NaN,NaN
1,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Load,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,1-3,Y,ADPBA1L,7.76,6.15,2,NaN,NaN,NaN,NaN
2,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Non-Scheduled,Hydro,Water,Renewable,Run of River,1-2,Y,ADPMH1,1.44,1,-,NaN,NaN,NaN,NaN
3,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Semi-Scheduled,Solar,Solar,Renewable,Photovoltaic Tracking Flat panel,"1-4, 5-9",Y,ADPPV1,24.75,19,4,NaN,NaN,NaN,NaN
4,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Non-Scheduled,Solar,Solar,Renewable,Photovoltaic Flat panel,1-2,Y,ADPPV2,0.2,0.2,-,NaN,NaN,NaN,NaN


In [28]:
print(nem_registration['Dispatch Type'].value_counts())
print(nem_registration['Fuel Source - Primary'].value_counts())
print(nem_registration['Fuel Source - Descriptor'].value_counts())
print(nem_registration['Region'].value_counts())

Dispatch Type
Generating Unit       493
Bidirectional Unit     30
Load                    9
Name: count, dtype: int64
Fuel Source - Primary
Fossil                                   160
Solar                                    119
Wind                                     104
Hydro                                     70
Battery storage                           39
Renewable/ Biomass / Waste                33
-                                          3
Renewable/ Biomass / Waste and Fossil      1
Name: count, dtype: int64
Fuel Source - Descriptor
Solar                              116
Wind                               105
Water                               70
Natural Gas                         66
Grid                                38
Black Coal                          35
Diesel                              19
Landfill Methane / Landfill Gas     18
Bagasse                             11
Coal Seam Methane                   10
Brown Coal                          10
Waste Coal Mine Gas 

In [16]:
## Dataset 2: Data from MMS Data Model
## base url is aemo_mms_url = "http://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/{}/MMSDM_{}_{}/MMSDM_Historical_Data_SQLLoader/DATA/{}.zip"
## Table DISPATCH.DISPATCH_UNIT_SCADA

## Download the latest data of DISPATCH.DISPATCH_UNIT_SCADA in MMS Data Model
## Time range: 2024-01 to 2024-12
## example url: https://nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/2024/MMSDM_2024_12/MMSDM_Historical_Data_SQLLoader/DATA/PUBLIC_ARCHIVE%23DISPATCH_UNIT_SCADA%23FILE01%23202412010000.zip

base_mms_url = "http://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/{}/MMSDM_{}_{}/MMSDM_Historical_Data_SQLLoader/DATA/{}.zip"

def download_dispatch_unit_scada_data(base_mms_url):

    for year in range(2024, 2026):
        for month in range(1, 13):
            month_str = f"{year}{month:02d}"
            if year >= 2024 and month > 12:
                continue
            if (year == 2024 and month >= 8) or year >= 2025:
                filename = f"PUBLIC_ARCHIVE%23DISPATCH_UNIT_SCADA%23FILE01%23{month_str}010000"
            else:
                filename = f"PUBLIC_DVD_DISPATCH_UNIT_SCADA_{month_str}010000"
            url = base_mms_url.format(year, year, f"{month:02d}", filename)

            zipname = f"data/aemo_data/DISPATCH_UNIT_SCADA_{month_str}.zip"
            csvname = f"data/aemo_data/DISPATCH_UNIT_SCADA_{month_str}.csv"

            try:
                if not os.path.exists(zipname):
                    response = requests.get(url, headers=headers)
                    response.raise_for_status()
                    with open(zipname, 'wb') as f:
                        f.write(response.content)
                    print(f"Downloaded {zipname}")
                else:
                    print(f"Skipping {zipname} - already exists")

                if not os.path.exists(csvname):
                    # Unzip the file and rename the CSV file
                    with zipfile.ZipFile(zipname, 'r') as zip_ref:
                        zip_ref.extractall("data/aemo_data")
                    print(f"Unzipped {zipname}")

                    # Rename the CSV file
                    filename = filename.replace('%23', '#')
                    os.rename(f"data/aemo_data/{filename}.CSV", csvname)
                    print(f"Renamed to {csvname}")
                else:
                    print(f"Skipping {csvname} - already exists")
            except requests.exceptions.RequestException as e:
                print(f"Failed to download {url}: {e}")

download_dispatch_unit_scada_data(base_mms_url)

Skipping data/aemo_data/DISPATCH_UNIT_SCADA_202401.zip - already exists
Skipping data/aemo_data/DISPATCH_UNIT_SCADA_202401.csv - already exists
Skipping data/aemo_data/DISPATCH_UNIT_SCADA_202402.zip - already exists
Skipping data/aemo_data/DISPATCH_UNIT_SCADA_202402.csv - already exists
Skipping data/aemo_data/DISPATCH_UNIT_SCADA_202403.zip - already exists
Skipping data/aemo_data/DISPATCH_UNIT_SCADA_202403.csv - already exists
Skipping data/aemo_data/DISPATCH_UNIT_SCADA_202404.zip - already exists
Skipping data/aemo_data/DISPATCH_UNIT_SCADA_202404.csv - already exists
Skipping data/aemo_data/DISPATCH_UNIT_SCADA_202405.zip - already exists
Skipping data/aemo_data/DISPATCH_UNIT_SCADA_202405.csv - already exists
Skipping data/aemo_data/DISPATCH_UNIT_SCADA_202406.zip - already exists
Skipping data/aemo_data/DISPATCH_UNIT_SCADA_202406.csv - already exists
Skipping data/aemo_data/DISPATCH_UNIT_SCADA_202407.zip - already exists
Skipping data/aemo_data/DISPATCH_UNIT_SCADA_202407.csv - already

In [21]:
## consider the data too large to be processed in memory, only use 2025-01 data for analysis

## prepare the data for analysis, delete the first and last line of the csv file
scada_df = pd.read_csv("data/aemo_data/DISPATCH_UNIT_SCADA_202501.csv", skiprows=1, skipfooter=1, engine='python')
scada_df.head()

,I,DISPATCH,UNIT_SCADA,1,SETTLEMENTDATE,DUID,SCADAVALUE,LASTCHANGED
0,D,DISPATCH,UNIT_SCADA,1,2025/01/01 00:05:00,ADPBA1,1.217,2025/01/01 00:00:13
1,D,DISPATCH,UNIT_SCADA,1,2025/01/01 00:05:00,ADPPV1,0.000,2025/01/01 00:00:13
2,D,DISPATCH,UNIT_SCADA,1,2025/01/01 00:05:00,AGLHAL,0.000,2025/01/01 00:00:13
3,D,DISPATCH,UNIT_SCADA,1,2025/01/01 00:05:00,AGLSOM,0.000,2025/01/01 00:00:13
4,D,DISPATCH,UNIT_SCADA,1,2025/01/01 00:05:00,ANGAST1,0.000,2025/01/01 00:00:13


In [22]:
## step 1: create a csv file with 100 lines as a screenshot of the data
scada_df_screenshot = scada_df.head(100)
scada_df_screenshot.to_csv("data/analysis/DISPATCH_UNIT_SCADA_202501_screenshot.csv", index=False)

## step 2: connect DISPATCH_UNIT_SCADA_202501.csv to NEM_Registration.csv using DUID
## merge the data
# scada_registration = pd.merge(scada_df, nem_registration, on='DUID', how='left')
# scada_registration.head()

,I,DISPATCH,UNIT_SCADA,1,SETTLEMENTDATE,DUID,SCADAVALUE,LASTCHANGED,Participant,Station Name,...,Technology Type - Descriptor,Units,Aggregation,Reg Cap generation (MW),Max Cap generation (MW),Max ROC/Min generation,Reg Cap consumption (MW),Max Cap consumption (MW),Max ROC/Min consumption,Comments
0,D,DISPATCH,UNIT_SCADA,1,2025/01/01 00:05:00,ADPBA1,1.217,2025/01/01 00:00:13,South Australian Water Corporation,Adelaide Desalination Plant,...,Battery and Inverter,"1,2-3",Y,7.76,6.15,4,7.0,6.0,4.0,NaN
1,D,DISPATCH,UNIT_SCADA,1,2025/01/01 00:05:00,ADPPV1,0.000,2025/01/01 00:00:13,South Australian Water Corporation,Adelaide Desalination Plant,...,Photovoltaic Tracking Flat panel,"1-4, 5-9",Y,24.75,19,4,NaN,NaN,NaN,NaN
2,D,DISPATCH,UNIT_SCADA,1,2025/01/01 00:05:00,AGLHAL,0.000,2025/01/01 00:00:13,EnergyAustralia Yallourn Pty Ltd,Hallett Power Station,...,Open Cycle Gas turbines (OCGT),1-13,Y,217,255,44,NaN,NaN,NaN,NaN
3,D,DISPATCH,UNIT_SCADA,1,2025/01/01 00:05:00,AGLSOM,0.000,2025/01/01 00:00:13,AGL Hydro Partnership,Somerton Power Station,...,Open Cycle Gas turbines (OCGT),1-4,Y,170,170,30,NaN,NaN,NaN,NaN
4,D,DISPATCH,UNIT_SCADA,1,2025/01/01 00:05:00,ANGAST1,0.000,2025/01/01 00:00:13,Snowy Hydro Limited,Angaston Power Station,...,Compression Reciprocating Engine,1-30,Y,50,50,14,NaN,NaN,NaN,NaN


In [ ]:
## step 3: filter the data columns 'Dispatch Type' = 'Generating Unit' and 'Bidirectional Unit' 
##         and SCADAVALUE need > 0
scada_df_gen = scada_df[(scada_df['SCADAVALUE'] > 0)]
print(scada_df_gen.shape)

(2025967, 8)


In [47]:
scada_df_gen.head()

,I,DISPATCH,UNIT_SCADA,1,SETTLEMENTDATE,DUID,SCADAVALUE,LASTCHANGED
0,D,DISPATCH,UNIT_SCADA,1,2025/01/01 00:05:00,ADPBA1,1.21700,2025/01/01 00:00:13
5,D,DISPATCH,UNIT_SCADA,1,2025/01/01 00:05:00,ARWF1,79.30000,2025/01/01 00:00:13
9,D,DISPATCH,UNIT_SCADA,1,2025/01/01 00:05:00,BANGOWF1,26.69719,2025/01/01 00:00:13
17,D,DISPATCH,UNIT_SCADA,1,2025/01/01 00:05:00,BASTYAN,76.11001,2025/01/01 00:00:13
26,D,DISPATCH,UNIT_SCADA,1,2025/01/01 00:05:00,BLOWERNG,55.10000,2025/01/01 00:00:13


In [49]:
## step 5: sum SCADAVALUE by DUID by day, for each DUID, calculate the daily total SCADAVALUE, other columns are the same
# SETTLEMENTDATE is in the format of 'YYYY-MM-DD HH:MM:SS'
scada_df_gen['SETTLEMENTDATE'] = pd.to_datetime(scada_df_gen['SETTLEMENTDATE'])
scada_df_gen['DATE'] = scada_df_gen['SETTLEMENTDATE'].dt.date

scada_daily = scada_df_gen.groupby(['DUID', 'DATE']).agg(
    SCADAVALUE_sum=('SCADAVALUE', 'sum')
).reset_index()
scada_daily.head()
    


/var/folders/2j/y447hdt109917s4bbcrzmh2m0000gn/T/ipykernel_29944/808518091.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scada_df_gen['SETTLEMENTDATE'] = pd.to_datetime(scada_df_gen['SETTLEMENTDATE'])
/var/folders/2j/y447hdt109917s4bbcrzmh2m0000gn/T/ipykernel_29944/808518091.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scada_df_gen['DATE'] = scada_df_gen['SETTLEMENTDATE'].dt.date


,DUID,DATE,SCADAVALUE_sum
0,ADPBA1,2025-01-01,84.260
1,ADPBA1,2025-01-02,128.208
2,ADPBA1,2025-01-03,215.221
3,ADPBA1,2025-01-04,124.450
4,ADPBA1,2025-01-05,81.853


In [55]:
## step 4: merge the data with NEM_Registration.csv
scada_daily_registration = pd.merge(scada_daily, nem_registration, on='DUID', how='left')
# filter the data columns 'Dispatch Type' = 'Generating Unit' and 'Bidirectional Unit'
scada_daily_registration = scada_daily_registration[scada_daily_registration['Dispatch Type'].isin(['Generating Unit', 'Bidirectional Unit'])]

# save to csv
scada_daily_registration.to_csv("data/analysis/DISPATCH_UNIT_SCADA_202501_daily.csv", index=False)
print("Saved to data/analysis/DISPATCH_UNIT_SCADA_202501_daily.csv")

Saved to data/analysis/DISPATCH_UNIT_SCADA_202501_daily.csv


In [57]:
# list and count unique values in 'Category', 'Fuel Source - Primary', 'Fuel Source - Descriptor',
#  'Technology Type - Primary', 'Technology Type - Descriptor', 

print(scada_daily_registration['Category'].value_counts())
print(scada_daily_registration['Fuel Source - Primary'].value_counts())
print(scada_daily_registration['Fuel Source - Descriptor'].value_counts())
print(scada_daily_registration['Technology Type - Primary'].value_counts())
print(scada_daily_registration['Technology Type - Descriptor'].value_counts())


Category
Market        10134
market           63
Non-Market       32
Market           31
Name: count, dtype: int64
Fuel Source - Primary
Solar                         3274
Wind                          2807
Fossil                        2119
Hydro                         1142
Battery storage                857
Renewable/ Biomass / Waste      61
Name: count, dtype: int64
Fuel Source - Descriptor
Solar                     3181
Wind                      2838
Water                     1142
Black Coal                 982
Grid                       826
Natural Gas                624
Brown Coal                 273
Coal Seam Methane           88
Solar                       62
Bagasse                     61
Waste Coal Mine Gas         53
Natural Gas / Fuel Oil      45
solar                       31
Diesel                      29
Natural Gas / Diesel        15
Kerosene                    10
Name: count, dtype: int64
Technology Type - Primary
Renewable     7223
Combustion    2180
Storage        8